In [24]:
import json
import pandas as pd

In [20]:
import json
import pandas as pd
from tqdm.notebook import tqdm

def split_json_to_csv(json_file, num_files):
    columns = ["id", "submitter", "authors", "title", "comments", "journal-ref", "doi", "report-no", "categories", "license", "abstract"]
    line_count_split = 2468402 // 4
    with open(json_file) as file:
        rows = []
        csv_count = 0
        for line in tqdm(file):
            obj = json.loads(line)
            rows.append(obj)
            if len(rows) > line_count_split:
                df = pd.DataFrame(rows, columns=columns)
                df.to_csv(f"arxiv_{csv_count}.csv", index=False)
                rows = []
                csv_count += 1
        
        if rows:
            df = pd.DataFrame(rows, columns=columns)
            df.to_csv(f"arxiv_{csv_count}.csv", index=False)
            
            
                
# Call the function to split the JSON file into 4 CSVs
split_json_to_csv("arxiv-metadata-oai-snapshot.json", 4)

2468403it [02:47, 14753.80it/s]


In [ ]:
file = 'arxiv-metadata-oai-snapshot.json'
with open(file) as train_file:
    d = json.load(train_file)

# converting json dataset from dictionary to dataframe
df = pd.DataFrame.from_dict(d, orient='index')
df.reset_index(level=0, inplace=True)

## Parse and relabel ArXiv categories

In [21]:
from bs4 import BeautifulSoup

def parse_html(filename):
    with open(filename, 'r') as file:
        soup = BeautifulSoup(file, 'html.parser')

    category_taxonomy_list = soup.find(id='category_taxonomy_list')
    if category_taxonomy_list is None:
        return {}

    categories = {}
    for h2_tag in category_taxonomy_list.find_all('h2', class_='accordion-head'):
        category_name = h2_tag.text.strip()
        
        # Find all sibling divs under the current h2 tag
        sibling_divs = h2_tag.find_next_siblings('div', class_='accordion-body')
        for div in sibling_divs:
            for h4_tag, p_tag in zip(div.find_all('h4'), div.find_all('p')):
                tag_text = h4_tag.text.strip()
                span_text = h4_tag.span.text.strip()[1:-1] if h4_tag.span else ""
                subtopic = tag_text.split()[0]
                description = p_tag.text.strip()
                
                categories[subtopic] = [category_name, span_text, description]

    return categories

categories_d = parse_html('arXivTaxonomy.html')

In [22]:
categories_d

{'cs.AI': ['Computer Science',
  'Artificial Intelligence',
  'Covers all areas of AI except Vision, Robotics, Machine Learning, Multiagent Systems, and Computation and\n              Language (Natural Language Processing), which have separate subject areas. In particular, includes Expert\n              Systems, Theorem Proving (although this may overlap with Logic in Computer Science), Knowledge\n              Representation, Planning, and Uncertainty in AI. Roughly includes material in ACM Subject Classes I.2.0,\n              I.2.1, I.2.3, I.2.4, I.2.8, and I.2.11.'],
 'cs.AR': ['Computer Science',
  'Hardware Architecture',
  'Covers systems organization and hardware architecture. Roughly includes material in ACM Subject Classes\n              C.0, C.1, and C.5.'],
 'cs.CC': ['Computer Science',
  'Computational Complexity',
  'Covers models of computation, complexity classes, structural complexity, complexity tradeoffs, upper and\n              lower bounds. Roughly includes mater

In [55]:
def categories_to_tags(cats):
    cats = cats.split()
    tags = set()
    for c in cats:
        if c in categories_d.keys():
            # arXiv0_df[arXiv0_df["categories"].str.contains("acc-phys")] but why tho
            tags.update([categories_d[c][0], categories_d[c][1]])
    return list(tags)

In [58]:
arXiv0_df = pd.read_csv("arxiv0.csv")
arXiv0_df["tags"] = arXiv0_df["categories"].apply(categories_to_tags)
arXiv0_df.to_csv("arxiv0_tags.csv")

/var/folders/16/w2tjk8ms4t322jq3ms1b49rh0000gn/T/ipykernel_85997/3422675069.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  arXiv0_df = pd.read_csv("arxiv0.csv")


In [79]:
arXiv1_df = pd.read_csv("arxiv1.csv")
arXiv1_df["tags"] = arXiv1_df["categories"].apply(categories_to_tags)
arXiv1_df.to_csv("arxiv1_tags.csv")

In [80]:
arXiv2_df = pd.read_csv("arxiv2.csv")
arXiv2_df["tags"] = arXiv2_df["categories"].apply(categories_to_tags)
arXiv2_df.to_csv("arxiv2_tags.csv")

In [81]:
arXiv3_df = pd.read_csv("arxiv3.csv")
arXiv3_df["tags"] = arXiv3_df["categories"].apply(categories_to_tags)
arXiv3_df.to_csv("arxiv3_tags.csv")